In [1]:
# This notebook, 'kernels' folder, nrkmeans.py and PreDeCon.py have to be in tudataset/tud_benchmark/ directory

# This means that you have to first download the TUDataset Repository e.g. with
# git clone https://github.com/chrsmrrs/tudataset.git

# Data Mining 20W group programming assignment

### Group 1
### dataset - IMDB_Binary
### algorithm - PreDeCon

In [3]:
import auxiliarymethods.auxiliary_methods as aux
from copy import deepcopy
from auxiliarymethods import datasets as dp
from auxiliarymethods.reader import tud_to_networkx
from matplotlib import pyplot as plt
import networkx as nx
from nrkmeans import NrKmeans
import numpy as np
import os
import pandas as pd
from PreDeCon import PreDeCon
from scipy.sparse import load_npz
import seaborn as sns
from sklearn.cluster import SpectralClustering, KMeans, AgglomerativeClustering, DBSCAN, FeatureAgglomeration, AffinityPropagation, SpectralClustering
from sklearn.datasets import make_blobs
from sklearn.decomposition import KernelPCA, TruncatedSVD, PCA
from sklearn.metrics import normalized_mutual_info_score
from sklearn.neighbors import LocalOutlierFactor


# Exploratory Data Analysis

In [4]:
# utility functions
def load_csv(path):
    return np.loadtxt(path, delimiter=";")

def load_sparse(path):
    return load_npz(path)

def select_from_list(l, indices):
    return [l[i] for i in indices]

## Example usage for gram matrix and sparse matrix with Weisfeiler-Lehman kernel

In [5]:
# # This code was used to get the results for each data set above:
# # Get some initial results for each data set
# # This will plot all representations and cluster these with Spectral Clustering and Subkmeans
# # In your case you might only want to run your data set

use_edge_labels = False
base_path = os.path.join("kernels", "without_labels")
dataset = "IMDB-BINARY"
print("Load from ", base_path)
nmis_kpca = {}
nmis_tsvd = {}
nmis_spec = {}
nmis_pred = {}

classes = dp.get_dataset(dataset)
nmis_kpca[dataset] = []
nmis_tsvd[dataset] = []
nmis_spec[dataset] = []
nmis_pred[dataset] = []

for iterations in range(1,6):
    # 0 taking just the nodelabels themselves into account; 
     # 1 considers nearest-neighbours, 2 one layer deeper and so on
    # play with this parameter to create a new kernel!
    print("##################################")
    print("Dataset ", dataset)
    print("Iteration ", iterations)
    print("##################################")

    #Gram Matrix for the Weisfeiler-Lehman subtree kernel
    gram = load_csv(os.path.join(base_path,f"{dataset}_gram_matrix_wl{iterations}.csv"))
    gram = aux.normalize_gram_matrix(gram)

    #Sparse Vectors for the Weisfeiler-Lehmann subtree kernel
    vec = load_sparse(os.path.join(base_path,f"{dataset}_vectors_wl{iterations}.npz"))
    print(gram.shape, vec.shape)

    

    
    



Load from  kernels/without_labels
##################################
Dataset  IMDB-BINARY
Iteration  1
##################################
(1000, 1000) (1000, 4009)
##################################
Dataset  IMDB-BINARY
Iteration  2
##################################
(1000, 1000) (1000, 8853)
##################################
Dataset  IMDB-BINARY
Iteration  3
##################################
(1000, 1000) (1000, 13717)
##################################
Dataset  IMDB-BINARY
Iteration  4
##################################
(1000, 1000) (1000, 18581)
##################################
Dataset  IMDB-BINARY
Iteration  5
##################################
(1000, 1000) (1000, 23445)


## Feature Engineering

## Dimensionality Reduction

In [17]:
for iterations in range(1,6):
    # 0 taking just the nodelabels themselves into account; 
     # 1 considers nearest-neighbours, 2 one layer deeper and so on
    # play with this parameter to create a new kernel!
    print("\n##################################")
    print("Dataset ", dataset)
    print("Iteration ", iterations)
    print("----------------------------------")

    #Gram Matrix for the Weisfeiler-Lehman subtree kernel
    gram = load_csv(os.path.join(base_path,f"{dataset}_gram_matrix_wl{iterations}.csv"))
    gram = aux.normalize_gram_matrix(gram)
    
    # apply PCA
    pca = PCA(n_components = 5)
    pca.fit(gram)
    print(pca.fit(gram))
    print(pca.explained_variance_ratio_)
    print(pca.singular_values_)
    
    

    


##################################
Dataset  IMDB-BINARY
Iteration  1
----------------------------------
PCA(n_components=5)
[0.41000563 0.18489879 0.13825097 0.07750716 0.05840446]
[168.17585294 112.93687425  97.6568748   73.12058405  63.47339693]

##################################
Dataset  IMDB-BINARY
Iteration  2
----------------------------------
PCA(n_components=5)
[0.36728508 0.16073165 0.12350552 0.09286731 0.06894332]
[97.25228815 64.33523313 56.39509995 48.90233716 42.13512323]

##################################
Dataset  IMDB-BINARY
Iteration  3
----------------------------------
PCA(n_components=5)
[0.31646818 0.14419196 0.12315295 0.10219165 0.05971233]
[71.05910538 47.96508769 44.32789587 40.37963756 30.86645245]

##################################
Dataset  IMDB-BINARY
Iteration  4
----------------------------------
PCA(n_components=5)
[0.27895136 0.15139078 0.1150423  0.08798912 0.05106174]
[58.60895804 43.17670318 37.63817407 32.91654424 25.07538283]

##################

## Clustering

In [66]:
for iterations in range(1,6):
    # 0 taking just the nodelabels themselves into account; 
     # 1 considers nearest-neighbours, 2 one layer deeper and so on
    # play with this parameter to create a new kernel!
    print("\n##################################")
    print("Dataset ", dataset)
    print("Iteration ", iterations)
    print("----------------------------------")

    #Gram Matrix for the Weisfeiler-Lehman subtree kernel
    gram = load_csv(os.path.join(base_path,f"{dataset}_gram_matrix_wl{iterations}.csv"))
    gram = aux.normalize_gram_matrix(gram)
    
    
    '''DBSCAN(eps=1, min_samples=2),
        DBSCAN(eps=1, min_samples=3),
        DBSCAN(eps=3, min_samples=1),
        DBSCAN(eps=3, min_samples=2),
        DBSCAN(eps=3, min_samples=3),
        FeatureAgglomeration(n_clusters=1001) # ValueError: Cannot extract more clusters than samples: 
                                                1001 clusters where given for a tree with 1000 leaves.
    '''
    
    cluster_algorithms = [
        # AgglomerativeClustering(),
        # DBSCAN(eps=1, min_samples=1),
        # FeatureAgglomeration(n_clusters=1000),
        # KMeans(n_clusters=500, max_iter=5),
        # AffinityPropagation(max_iter=50, affinity='euclidean'), ???????
        # SpectralClustering(affinity='precomputed')     ???????
    ]
    
    for algorithm in cluster_algorithms:
        print('algorithm      - ', algorithm)
        clustering = algorithm.fit(gram)
        nmi_score = normalized_mutual_info_score(clustering.labels_, classes)
        print('clustering nmi - ', nmi_score)
        print("----------------------------------")
    
    
    


##################################
Dataset  IMDB-BINARY
Iteration  1
----------------------------------
algorithm      -  SpectralClustering(affinity='precomputed')
clustering nmi -  0.08932857822930954
----------------------------------

##################################
Dataset  IMDB-BINARY
Iteration  2
----------------------------------
algorithm      -  SpectralClustering(affinity='precomputed')
clustering nmi -  0.08808308999142592
----------------------------------

##################################
Dataset  IMDB-BINARY
Iteration  3
----------------------------------
algorithm      -  SpectralClustering(affinity='precomputed')
clustering nmi -  0.08799650871638601
----------------------------------

##################################
Dataset  IMDB-BINARY
Iteration  4
----------------------------------
algorithm      -  SpectralClustering(affinity='precomputed')
clustering nmi -  0.08737337620083317
----------------------------------

##################################
Dataset 

## Outlier Detection

In [7]:
for iterations in range(1,6):
    # 0 taking just the nodelabels themselves into account; 
     # 1 considers nearest-neighbours, 2 one layer deeper and so on
    # play with this parameter to create a new kernel!
    print("\n##################################")
    print("Dataset ", dataset)
    print("Iteration ", iterations)
    print("----------------------------------")

    #Gram Matrix for the Weisfeiler-Lehman subtree kernel
    gram = load_csv(os.path.join(base_path,f"{dataset}_gram_matrix_wl{iterations}.csv"))
    gram = aux.normalize_gram_matrix(gram)
    
    
    
    lof = LocalOutlierFactor(n_neighbors=2)
    res = lof.fit_predict(gram)
    print(lof)
    print(lof.negative_outlier_factor_)
    
    
    



##################################
Dataset  IMDB-BINARY
Iteration  1
----------------------------------
LocalOutlierFactor(n_neighbors=2)
[-1.03601899e+00 -1.28933770e+00 -1.22041252e+00 -1.00604777e+00
 -1.00000000e+00 -1.03497723e+00 -1.00000000e+00 -1.00000000e+00
 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00
 -2.30571504e+10 -1.73927013e+10 -9.49913838e-01 -1.00000000e+00
 -1.16823438e+00 -1.00000000e+00 -1.00000000e+00 -1.25538697e+00
 -9.64683488e-01 -2.61139578e+00 -1.38619593e+10 -1.69725299e+00
 -1.00000000e+00 -2.20374635e+00 -4.73917195e+09 -1.28933770e+00
 -1.00000000e+00 -1.00000000e+00 -1.00297849e+00 -1.03653703e+00
 -6.43902763e+09 -1.40043427e+10 -1.00000000e+00 -1.00000000e+00
 -1.00000000e+00 -1.00000000e+00 -1.42894650e+00 -1.00000000e+00
 -1.21023436e+00 -1.18831593e+00 -1.00000000e+00 -1.00000000e+00
 -1.00000000e+00 -1.02264056e+00 -1.00000000e+00 -1.00000000e+00
 -1.00000000e+00 -1.72090436e+10 -9.92947231e-01 -1.00000000e+00
 -1.00000000e+00

AttributeError: 'numpy.ndarray' object has no attribute 'negative_outlier_factor_'

## Visualisation

## Visualize and interpret your results

IMDB-BINARY data set.

In [30]:
def visualize(G, color=None, figsize=(5,5)):
    plt.figure(figsize=figsize)
    plt.xticks([])
    plt.yticks([])
    nx.draw_networkx(G, 
                     pos=nx.spring_layout(G, seed=42),
                     with_labels=True,
                     node_color=color,
                     cmap="Set2")
    plt.show();

In [31]:
base_path = os.path.join("kernels", "without_labels")
ds_name = "IMDB-BINARY"
classes = dp.get_dataset(ds_name)
G = tud_to_networkx(ds_name)
print(f"Number of graphs in data set is {len(G)}")
print(f"Number of classes {len(set(classes.tolist()))}")

Number of graphs in data set is 1000
Number of classes 2
